In [16]:
import numpy as np

In [17]:
# relies on chopping off column headers ahead of time. 
artists = np.loadtxt('data/artists.csv', dtype='string', delimiter=',')
print artists
profiles = np.loadtxt('data/profiles_fixed.csv', dtype='string', delimiter=',')
print profiles.shape

# fixing erroneous, irrelevant ages like 1002
max_age = 120
# total = 0
# count = 0
for p in profiles:
    if p[2] != '' and (int(p[2]) >= max_age or int(p[2]) < 0):
        # 22 is the average age of those who put down ages within a reasonable range
        p[2] = '22'
#     if p[2] != '':
#         total += int(p[2])
#         count += 1
# print float(total) / count

[['03098741-08b3-4dd7-b3f6-1b0bfa2c879c' 'Liars']
 ['69c4cc43-8163-41c5-ac81-30946d27bb69' 'CunninLynguists']
 ['7a2e6b55-f149-4e74-be6a-30a1b1a387bb' 'The Desert Sessions']
 ..., 
 ['8974da95-e631-45aa-8fd7-aa0c2795f997' 'Harry Gregson-Williams']
 ['8067c102-4996-42bc-9980-06ce2e644eae' 'Soul Coughing']
 ['39c2a93d-9afa-4a22-9bba-c087ab056e1c' 'Jefferson Airplane']]
(233286, 4)


In [18]:
art_dict = {}
count = 0
for a in artists:
    art_dict[a[0]] = count
    count += 1
# print art_dict
prof_dict = {}
count = 0
for a in profiles:
    prof_dict[a[0]] = count
    count += 1

In [19]:
nation_dict = {}
count = 0
for p in profiles:
    if p[3] not in nation_dict:
        nation_dict[p[3]] = None
num_nations = len(nation_dict)
count = 0
for key in nation_dict.keys():
    nation_dict[key] = np.zeros(num_nations)
    nation_dict[key][count] = 1
    count += 1

In [20]:
train = np.loadtxt('data/train_fixed.csv', dtype='string', delimiter=',')
print train.shape

(4154804, 3)


In [21]:
art_counts = np.zeros((artists.shape[0], 3 + max_age + 1 + len(nation_dict)))
artist_counts = [0] * 2000
# count = 0
for t in train:
    
    artist = art_dict[t[1]]
    
    p = prof_dict[t[0]]
    
    one_hot_gender = np.zeros(3)
    if profiles[p][1] == 'f':
        one_hot_gender[0] = 1
    elif profiles[p][1] == 'm':
        one_hot_gender[1] = 1
    else:
        one_hot_gender[2] = 1

    one_hot_age = np.zeros(max_age + 1)
    if profiles[p][2] == '':
        one_hot_age[max_age] = 1
    else:
        one_hot_age[int(profiles[p][2])] = 1
        
    one_hot_nation = nation_dict[profiles[p][3]]
    
    user_info = np.concatenate((one_hot_gender, one_hot_age, one_hot_nation))
    
    art_counts[artist] += user_info
    artist_counts[artist] += 1
#     count += 1


In [22]:
print art_counts[-1]
print artist_counts[-1]

[  6.04000000e+02   1.52600000e+03   1.93000000e+02   0.00000000e+00
   3.00000000e+00   2.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   3.00000000e+00   1.00000000e+00   1.50000000e+01   3.50000000e+01
   8.50000000e+01   1.04000000e+02   1.74000000e+02   2.08000000e+02
   1.68000000e+02   1.60000000e+02   1.45000000e+02   1.29000000e+02
   9.20000000e+01   8.40000000e+01   7.90000000e+01   4.70000000e+01
   3.60000000e+01   2.90000000e+01   3.20000000e+01   2.50000000e+01
   2.20000000e+01   1.70000000e+01   1.10000000e+01   2.00000000e+01
   6.00000000e+00   9.00000000e+00   1.30000000e+01   2.00000000e+00
   8.00000000e+00   5.00000000e+00   3.00000000e+00   2.00000000e+00
   5.00000000e+00   1.00000000e+00   9.00000000e+00   1.10000000e+01
   6.00000000e+00   3.00000000e+00   3.00000000e+00   5.00000000e+00
   5.00000000e+00   6.00000000e+00

In [23]:
for i in range(2000):
    art_counts[i] /= artist_counts[i]

In [24]:
np.save('art_counts_small', art_counts)

In [25]:
def centroid_dist(x, c):
        sub = x - c
        sum_squares = np.square(sub)
        return sum(sum_squares)

def closest_centroid(cents, x):
        distances = []
        for c in cents:
            distances.append(centroid_dist(x, c))
        return distances.index(min(distances))

In [26]:
cents = np.load('art_small_cluster.npy')
print cents

[[  2.63555419e-01   6.61543444e-01   7.49011370e-02 ...,   1.75746924e-04
    2.10522813e-04   1.16686114e-04]
 [  1.60047399e-01   7.36738911e-01   1.03213690e-01 ...,   9.31967595e-05
    3.03975718e-04   0.00000000e+00]
 [  3.97323369e-01   5.55975648e-01   4.67009828e-02 ...,   0.00000000e+00
    4.16557157e-04   0.00000000e+00]
 ..., 
 [  1.45149924e-01   7.95820498e-01   5.90295771e-02 ...,   3.90617106e-04
    1.75592625e-04   3.57589844e-05]
 [  1.55277214e-01   7.56058146e-01   8.86646404e-02 ...,   9.91912180e-05
    2.48454266e-04   0.00000000e+00]
 [  3.25946475e-01   5.94921634e-01   7.91318904e-02 ...,   9.09949788e-05
    9.78355554e-05   7.86277074e-05]]


In [27]:
t_a_c = []
count = 0
for t in train:
    artist = art_dict[t[1]]
    t_a_c.append([])
    t_a_c[count].append(t[0])
    t_a_c[count].append(closest_centroid(cents, art_counts[artist]))
    t_a_c[count].append(t[2])
    
    count += 1
    if count % 10000 == 0:
        print float(count) / 1048575
#     if count % 100 == 0:
#         print art_counts[artist]
#         print closest_centroid(cents, art_counts[artist])

0.00953675225902
0.019073504518
0.0286102567771
0.0381470090361
0.0476837612951
0.0572205135541
0.0667572658131
0.0762940180721
0.0858307703312
0.0953675225902
0.104904274849
0.114441027108
0.123977779367
0.133514531626
0.143051283885
0.152588036144
0.162124788403
0.171661540662
0.181198292921
0.19073504518
0.200271797439
0.209808549698
0.219345301957
0.228882054216
0.238418806475
0.247955558734
0.257492310993
0.267029063253
0.276565815512
0.286102567771
0.29563932003
0.305176072289
0.314712824548
0.324249576807
0.333786329066
0.343323081325
0.352859833584
0.362396585843
0.371933338102
0.381470090361
0.39100684262
0.400543594879
0.410080347138
0.419617099397
0.429153851656
0.438690603915
0.448227356174
0.457764108433
0.467300860692
0.476837612951
0.48637436521
0.495911117469
0.505447869728
0.514984621987
0.524521374246
0.534058126505
0.543594878764
0.553131631023
0.562668383282
0.572205135541
0.5817418878
0.591278640059
0.600815392318
0.610352144577
0.619888896836
0.629425649095
0.6389

In [28]:
train_art_clusters = np.array(t_a_c)
print train_art_clusters

[['eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03' '16' '554']
 ['44ce793a6cd9d20f13f4a576a818ef983314bb5d' '29' '81']
 ['da9cf3f557161d54b76f24db64be9cc76db008e3' '9' '708']
 ..., 
 ['effb7b65d5edf0ff8753b07994f07d4b00f8c484' '3' '27']
 ['3f39b84f249adea34282581b2bbc627d9ada7701' '3' '1205']
 ['1de2692ba7a857e0a6bb27898ecdad37db202b71' '38' '553']]


In [29]:
np.save('train_art_clusters', train_art_clusters)

In [ ]:
train_art_clusters = np.load('train_art_clusters.npy')

In [ ]:
print train_art_clusters[:100]

In [13]:
pro_counts = np.zeros((profiles.shape[0], 50))
profile_counts = [0] * profiles.shape[0]
count = 0
print train_art_clusters.shape
for t in train_art_clusters:
    
    p = prof_dict[t[0]]
    
    a = int(t[1])
    
    pro_counts[p][a] += int(t[2])
    profile_counts[p] += 1
    
    if count % 10000 == 0:
        print float(count) / 1048575
    count += 1

(1048575, 3)
0.0
0.00953675225902
0.019073504518
0.0286102567771
0.0381470090361
0.0476837612951
0.0572205135541
0.0667572658131
0.0762940180721
0.0858307703312
0.0953675225902
0.104904274849
0.114441027108
0.123977779367
0.133514531626
0.143051283885
0.152588036144
0.162124788403
0.171661540662
0.181198292921
0.19073504518
0.200271797439
0.209808549698
0.219345301957
0.228882054216
0.238418806475
0.247955558734
0.257492310993
0.267029063253
0.276565815512
0.286102567771
0.29563932003
0.305176072289
0.314712824548
0.324249576807
0.333786329066
0.343323081325
0.352859833584
0.362396585843
0.371933338102
0.381470090361
0.39100684262
0.400543594879
0.410080347138
0.419617099397
0.429153851656
0.438690603915
0.448227356174
0.457764108433
0.467300860692
0.476837612951
0.48637436521
0.495911117469
0.505447869728
0.514984621987
0.524521374246
0.534058126505
0.543594878764
0.553131631023
0.562668383282
0.572205135541
0.5817418878
0.591278640059
0.600815392318
0.610352144577
0.619888896836
0.62

In [ ]:
# for i in range(len(pro_counts)):
#     if profile_counts[i] > 1:
#         pro_counts[i] /= profile_counts[i]

In [14]:
print pro_counts[3022]

[   0.    0.    0.    0.    0.    0.    0.  139.    0.    0.  150.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.  104.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.]


In [15]:
np.save('pro_counts', pro_counts)